In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/weather_bigdata_contest/code/

/content/drive/MyDrive/weather_bigdata_contest/code


In [3]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 5.4 kB/s 


In [4]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from catboost import CatBoostRegressor

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
path = '../data/'
#path = '/content/drive/MyDrive/final weather/data/'
wea_sale_ohe = pd.read_csv(path+'weather_sale_ohe.csv')
wea_sns_ohe = pd.read_csv(path+'weather_sns_ohe.csv')

## Ensemble Model


### 데이터 전처리

In [6]:
def sale_dataset(ohe_df):
    # 상품 목록을 정의합니다. 
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()
    # 상품 및 날짜로 데이터를 정렬합니다.
    ohe_df.sort_values(goods+['날짜'], inplace=True)
    # 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
    ohe_df.index = range(0, len(ohe_df))

    # 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled_ohe = sc.fit_transform(ohe_df.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

    # 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
    # scale된 데이터를 병합해서 total data를 생성합니다.
    # 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
    total = np.hstack([ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                    scaled_ohe,
                    ohe_df.loc[:,'20대 여성 판매량(개)':'60대 남성 판매량(개)'].values])

    # 가열식 가습기 판매 데이터 개수의 80% 입니다.
    index =int(len(ohe_df[ohe_df['소분류_가열식 가습기']==1]) * 0.8)

    # 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
    train = total[ohe_df['소분류_가열식 가습기']==1][:index]
    test = total[ohe_df['소분류_가열식 가습기']==1][index:]

    # 전체 상품 목록 리스트
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
    for good in goods[1:]:
        index =int(len(ohe_df[ohe_df[good]==1]) * 0.8)
        train = np.vstack([train, total[ohe_df[good]==1][:index]])
        test = np.vstack([test, total[ohe_df[good]==1][index:]])

    day = 20

    # X_train, y_train 생성

    df = train[train[:, 0]==1] 
    X_train = df[day-1:, :-10]
    y_train = df[day-1:, -10:]

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = train[train[:, good_index]==1] 
      X_train = np.vstack([X_train, df[day-1:, :-10]])
      y_train = np.vstack([y_train, df[day-1:, -10:]])

    df = test[test[:, 0]==1] 
    X_test = df[day-1:, :-10]
    y_test = df[day-1:, -10:]

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = test[test[:, good_index]==1] 

      # 해당 판매 데이터를 day 기준으로 분할합니다.(20일)
      X_test = np.vstack([X_test, df[day-1:, :-10]])
      y_test = np.vstack([y_test, df[day-1:, -10:]])

    return X_train, X_test, y_train, y_test

In [7]:
def sns_dataset(ohe_df):
    # 상품 목록을 정의합니다. 
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 상품 및 날짜로 데이터를 정렬합니다.
    ohe_df.sort_values(goods+['날짜'], inplace=True)

    # 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
    ohe_df.index = range(0, len(ohe_df))

    # 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.
    sc = MinMaxScaler(feature_range=(0, 1))
    scaled_ohe = sc.fit_transform(ohe_df.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

    # 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
    # scale된 데이터를 병합해서 total data를 생성합니다.
    # 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
    total = np.hstack([ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                    scaled_ohe,
                    ohe_df.loc[:,'SNS언급량'].values.reshape(-1, 1)])

    # 가열식 가습기 판매 데이터 개수의 80% 입니다.
    index =int(len(ohe_df[ohe_df['소분류_가열식 가습기']==1]) * 0.8)

    # 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
    train = total[ohe_df['소분류_가열식 가습기']==1][:index]
    test = total[ohe_df['소분류_가열식 가습기']==1][index:]

    # 전체 상품 목록 리스트
    goods = ohe_df.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

    # 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
    for good in goods[1:]:
        index =int(len(ohe_df[ohe_df[good]==1]) * 0.8)
        train = np.vstack([train, total[ohe_df[good]==1][:index]])
        test = np.vstack([test, total[ohe_df[good]==1][index:]])

    day = 20

    df = train[train[:, 0]==1] 
    X_train = df[day-1:, :-1]
    y_train = df[day-1:, -1].reshape(-1,1)

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = train[train[:, good_index]==1] 
      X_train = np.vstack([X_train, df[day-1:, :-1]])
      y_train = np.vstack([y_train, df[day-1:, -1].reshape(-1,1)])

    df = test[test[:, 0]==1] 
    X_test = df[day-1:, :-1]
    y_test = df[day-1:, -1].reshape(-1,1)

    # good_index는 상품의 인덱스를 의미합니다.
    # 총 '126'개의 상품이 있습니다. 

    for good_index in range(1, 126):
      # 해당 인덱스의 상품에 대한 판매 데이터만 추출합니다.
      df = test[test[:, good_index]==1] 

      # 해당 판매 데이터를 day 기준으로 분할합니다.(20일)
      X_test = np.vstack([X_test, df[day-1:, :-1]])
      y_test = np.vstack([y_test, df[day-1:, -1].reshape(-1,1)])

    return X_train, X_test, y_train.reshape(-1), y_test.reshape(-1)

### 1) LGBMRegressor




#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [8]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [ ]:
lgbm_reg = LGBMRegressor(n_estimators = 300)
mo_reg_lgbm = MultiOutputRegressor(lgbm_reg)

eval_set = [(X_test, y_test)]

mo_reg_lgbm.fit(X_train, y_train)

pred = mo_reg_lgbm.predict(X_test)

lgbm_sale_mse = mean_squared_error(y_test, pred)
lgbm_sale_mae = mean_absolute_error(y_test, pred)
print('LGBMRegressor의 판매량 예측 MSE: ', lgbm_sale_mse)
print('LGBMRegressor의 판매량 예측 RMSE: ', np.sqrt(lgbm_sale_mse))
print('LGBMRegressor의 판매량 예측 MAE: ', lgbm_sale_mae)


LGBMRegressor의 판매량 예측 MSE:  439.5142314168567
LGBMRegressor의 판매량 예측 RMSE:  20.96459471148576
LGBMRegressor의 판매량 예측 MAE:  7.958411629060916


In [ ]:
from sklearn.externals import joblib

joblib.dump(mo_reg_lgbm, '../data/mo_reg_lgbm_sale.pkl')

['../data/mo_reg_lgbm_sale.pkl']

In [9]:
load_lgbm = joblib.load('../data/mo_reg_lgbm_sale.pkl')

lgbm_sale_pred = load_lgbm.predict(X_test)

lgbm_sale_mse = mean_squared_error(y_test, lgbm_sale_pred)
lgbm_sale_mae = mean_absolute_error(y_test, lgbm_sale_pred)
print('LGBMRegressor의 판매량 예측 MSE: ', lgbm_sale_mse)
print('LGBMRegressor의 판매량 예측 RMSE: ', np.sqrt(lgbm_sale_mse))
print('LGBMRegressor의 판매량 예측 MAE: ', lgbm_sale_mae)

LGBMRegressor의 판매량 예측 MSE:  439.5142314168567
LGBMRegressor의 판매량 예측 RMSE:  20.96459471148576
LGBMRegressor의 판매량 예측 MAE:  7.958411629060916


#### (2) SNS 언급량 예측

In [10]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [ ]:
lgbm_reg = LGBMRegressor(n_estimators = 300)
eval_set = [(X_test, y_test)]
lgbm_reg.fit(X_train, y_train, eval_metric = 'mean_squared_error',
             eval_set = eval_set, verbose = False)
lgbm_sns_pred = lgbm_reg.predict(X_test)

lgbm_sns_mse = mean_squared_error(lgbm_sns_pred, y_test)
lgbm_sns_mae = mean_absolute_error(lgbm_sns_pred, y_test)
print('LGBMRegressor의 SNS 언급량 예측 MSE:', lgbm_sns_mse)
print('LGBMRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(lgbm_sns_mse))
print('LGBMRegressor의 SNS 언급량 예측 MAE:', lgbm_sns_mae)


LGBMRegressor의 SNS 언급량 예측 MSE: 674.1798275828507
LGBMRegressor의 SNS 언급량 예측 RMSE: 25.964973090354835
LGBMRegressor의 SNS 언급량 예측 MAE: 10.479638969313406


In [ ]:
from sklearn.externals import joblib

joblib.dump(lgbm_reg, '../data/lgbm_reg_sns.pkl')

['../data/lgbm_reg_sns.pkl']

In [11]:
from sklearn.externals import joblib

load_lgbm = joblib.load('../data/lgbm_reg_sns.pkl')

lgbm_sns_pred = load_lgbm.predict(X_test)

lgbm_sns_mse = mean_squared_error(lgbm_sns_pred, y_test)
lgbm_sns_mae = mean_absolute_error(lgbm_sns_pred, y_test)
print('LGBMRegressor의 SNS 언급량 예측 MSE:', lgbm_sns_mse)
print('LGBMRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(lgbm_sns_mse))
print('LGBMRegressor의 SNS 언급량 예측 MAE:', lgbm_sns_mae)

LGBMRegressor의 SNS 언급량 예측 MSE: 674.1798275828507
LGBMRegressor의 SNS 언급량 예측 RMSE: 25.964973090354835
LGBMRegressor의 SNS 언급량 예측 MAE: 10.479638969313406


### 2) CatBoostRegressor




#### (1) 연령, 성별 카테고리별 총 판매량 예측

In [12]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [ ]:
cb_reg = CatBoostRegressor(n_estimators = 300)
mo_reg_cb = MultiOutputRegressor(cb_reg)

eval_set = [(X_test, y_test)]

mo_reg_cb.fit(X_train, y_train)

pred = mo_reg_cb.predict(X_test)

cb_sale_mse = mean_squared_error(y_test, pred)
cb_sale_mae = mean_absolute_error(y_test, pred)
print('CatBoostRegressor의 판매량 예측 MSE: ', cb_sale_mse)
print('CatBoostRegressor의 판매량 예측 RMSE: ', np.sqrt(cb_sale_mse))
print('CatBoostRegressor의 판매량 예측 MAE: ', cb_sale_mae)

Learning rate set to 0.20735
0:	learn: 48.3342575	total: 61.3ms	remaining: 18.3s
1:	learn: 42.8550560	total: 72.8ms	remaining: 10.8s
2:	learn: 38.9694495	total: 83.7ms	remaining: 8.28s
3:	learn: 36.2307462	total: 94.4ms	remaining: 6.98s
4:	learn: 34.1473449	total: 105ms	remaining: 6.2s
5:	learn: 32.6215383	total: 116ms	remaining: 5.66s
6:	learn: 31.4767886	total: 127ms	remaining: 5.29s
7:	learn: 30.6037425	total: 139ms	remaining: 5.08s
8:	learn: 29.8122938	total: 152ms	remaining: 4.92s
9:	learn: 29.2519906	total: 163ms	remaining: 4.72s
10:	learn: 28.6980474	total: 173ms	remaining: 4.56s
11:	learn: 28.1998437	total: 188ms	remaining: 4.52s
12:	learn: 27.7657453	total: 199ms	remaining: 4.4s
13:	learn: 27.3682571	total: 210ms	remaining: 4.29s
14:	learn: 27.0158022	total: 221ms	remaining: 4.2s
15:	learn: 26.7106099	total: 232ms	remaining: 4.11s
16:	learn: 26.3680864	total: 243ms	remaining: 4.04s
17:	learn: 26.0609905	total: 254ms	remaining: 3.97s
18:	learn: 25.7731861	total: 265ms	remaining

In [ ]:
from sklearn.externals import joblib

joblib.dump(mo_reg_cb, '../data/mo_reg_cb_sale.pkl')

['../data/mo_reg_cb_sale.pkl']

In [13]:
load_cb = joblib.load('../data/mo_reg_cb_sale.pkl')

cb_sale_pred = load_cb.predict(X_test)

cb_sale_mse = mean_squared_error(y_test, cb_sale_pred)
cb_sale_mae = mean_absolute_error(y_test, cb_sale_pred)
print('CatBoostRegressor의 판매량 예측 MSE: ', cb_sale_mse)
print('CatBoostRegressor의 판매량 예측 RMSE: ', np.sqrt(cb_sale_mse))
print('CatBoostRegressor의 판매량 예측 MAE: ', cb_sale_mae)

CatBoostRegressor의 판매량 예측 MSE:  437.20838539796597
CatBoostRegressor의 판매량 예측 RMSE:  20.909528579046587
CatBoostRegressor의 판매량 예측 MAE:  7.981197987829626


#### (2) SNS 언급량 예측

In [14]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [ ]:
cb_reg = CatBoostRegressor(n_estimators = 300)
eval_set = [(X_test, y_test)]
cb_reg.fit(X_train, y_train, verbose = True)
cb_sns_pred = cb_reg.predict(X_test)

cb_sns_mse = mean_squared_error(cb_sns_pred, y_test)
cb_sns_mae = mean_absolute_error(cb_sns_pred, y_test)
print('LGBMRegressor의 SNS 언급량 예측 MSE:', cb_sns_mse)
print('LGBMRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(cb_sns_mse))
print('LGBMRegressor의 SNS 언급량 예측 MAE:', cb_sns_mae)


Learning rate set to 0.208218
0:	learn: 150.8000343	total: 16.1ms	remaining: 4.82s
1:	learn: 125.8514785	total: 26.5ms	remaining: 3.95s
2:	learn: 106.6706386	total: 39.8ms	remaining: 3.94s
3:	learn: 92.1257699	total: 56.3ms	remaining: 4.17s
4:	learn: 80.8982657	total: 71.8ms	remaining: 4.24s
5:	learn: 72.4510310	total: 82.1ms	remaining: 4.02s
6:	learn: 65.8879266	total: 92.4ms	remaining: 3.87s
7:	learn: 60.7861372	total: 103ms	remaining: 3.76s
8:	learn: 56.8545427	total: 115ms	remaining: 3.71s
9:	learn: 53.7361192	total: 125ms	remaining: 3.63s
10:	learn: 51.1800554	total: 140ms	remaining: 3.68s
11:	learn: 49.1358576	total: 151ms	remaining: 3.61s
12:	learn: 47.3674038	total: 161ms	remaining: 3.56s
13:	learn: 45.7206247	total: 172ms	remaining: 3.52s
14:	learn: 44.3540990	total: 183ms	remaining: 3.47s
15:	learn: 43.1737506	total: 193ms	remaining: 3.43s
16:	learn: 41.9847180	total: 206ms	remaining: 3.44s
17:	learn: 41.0257864	total: 219ms	remaining: 3.44s
18:	learn: 40.1338940	total: 230ms

In [ ]:
from sklearn.externals import joblib

joblib.dump(cb_reg, '../data/cb_reg_sns.pkl')

['../data/cb_reg_sns.pkl']

In [15]:
load_cb = joblib.load('../data/cb_reg_sns.pkl')

cb_sns_pred = load_cb.predict(X_test)

cb_sns_mse = mean_squared_error(y_test, cb_sns_pred)
cb_sns_mae = mean_absolute_error(y_test, cb_sns_pred)
print('CatBoostRegressor의 판매량 예측 MSE: ', cb_sns_mse)
print('CatBoostRegressor의 판매량 예측 RMSE: ', np.sqrt(cb_sns_mse))
print('CatBoostRegressor의 판매량 예측 MAE: ', cb_sns_mae)

CatBoostRegressor의 판매량 예측 MSE:  650.9868435318208
CatBoostRegressor의 판매량 예측 RMSE:  25.514443821722253
CatBoostRegressor의 판매량 예측 MAE:  10.524364601335625


## 3) DNN
- 딥러닝의 기본형태인 DNN(Deep neural network)을 통해 날씨에 따른 집단별 판매량 및 SNS 언급량을 예측합니다,

In [16]:
import tensorflow as tf
from tensorflow import keras

In [17]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


### (1) 연령, 성별 카테고리별 총 판매량 예측
- 날씨에 따른 성별 및 연령별 판매량을 예측합니다.

In [18]:
X_train, X_test, y_train, y_test = sale_dataset(wea_sale_ohe)

In [ ]:
with tf.device('/device:GPU:0'):
    dnn = keras.Sequential([
                        keras.layers.Dense(64, input_dim = 134),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(64),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(10)
])
    adam = keras.optimizers.Adam(learning_rate = 0.01)
    dnn.compile(loss = 'mean_squared_error', optimizer = adam)
    dnn.fit(X_train, y_train, epochs = 300, validation_data = (X_test, y_test), batch_size = 1000)

    pred = dnn.predict(X_test)

    dnn_sale_mse = mean_squared_error(pred, y_test)
    dnn_sale_mae = mean_absolute_error(pred, y_test)
    print('DNN의 판매량 예측 MSE: ', dnn_sale_mse)
    print('DNN의 판매량 예측 RMSE: ', np.sqrt(dnn_sale_mse))
    print('DNN의 판매량 예측 MAE: ', dnn_sale_mae)

Epoch 1/300
70/70 [==============================] - 4s 7ms/step - loss: 1491.0145 - val_loss: 1817.6665
Epoch 2/300
70/70 [==============================] - 0s 5ms/step - loss: 488.1708 - val_loss: 1459.0414
Epoch 3/300
70/70 [==============================] - 0s 4ms/step - loss: 438.6545 - val_loss: 1065.1495
Epoch 4/300
70/70 [==============================] - 0s 5ms/step - loss: 429.8164 - val_loss: 751.4220
Epoch 5/300
70/70 [==============================] - 0s 5ms/step - loss: 429.4751 - val_loss: 634.8952
Epoch 6/300
70/70 [==============================] - 0s 5ms/step - loss: 420.9529 - val_loss: 510.9064
Epoch 7/300
70/70 [==============================] - 0s 5ms/step - loss: 412.6216 - val_loss: 446.8965
Epoch 8/300
70/70 [==============================] - 0s 5ms/step - loss: 413.4523 - val_loss: 452.6464
Epoch 9/300
70/70 [==============================] - 0s 5ms/step - loss: 410.6388 - val_loss: 465.5059
Epoch 10/300
70/70 [==============================] - 0s 5ms/step - l

In [ ]:
dnn.save('../data/dnn_sale.h5')

In [19]:
dnn = keras.models.load_model('../data/dnn_sale.h5')

dnn_sale_pred = dnn.predict(X_test)

dnn_sale_mse = mean_squared_error(dnn_sale_pred, y_test)
dnn_sale_mae = mean_absolute_error(dnn_sale_pred, y_test)
print('DNN의 판매량 예측 MSE: ', dnn_sale_mse)
print('DNN의 판매량 예측 RMSE: ', np.sqrt(dnn_sale_mse))
print('DNN의 판매량 예측 MAE: ', dnn_sale_mae)

DNN의 판매량 예측 MSE:  441.1367926895761
DNN의 판매량 예측 RMSE:  21.00325671627084
DNN의 판매량 예측 MAE:  7.805166339456248


### (2) SNS 언급량 예측
- 날씨에 따른 SNS 언급량을 예측합니다.

In [20]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [ ]:
with tf.device('/device:GPI:0'):
    dnn = keras.Sequential([
                        keras.layers.Dense(64, input_dim = 134),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(64),
                        keras.layers.BatchNormalization(),
                        keras.layers.ReLU(),
                        keras.layers.Dense(1)
])

    dnn.compile(loss = 'mean_squared_error', optimizer = 'adam')

    dnn.fit(X_train, y_train, epochs = 300, validation_data = (X_test, y_test), batch_size = 1000)

    pred = dnn.predict(X_test)

    dnn_sns_mse = mean_squared_error(pred, y_test)
    dnn_sns_mae = mean_absolute_error(pred, y_test)
    print('DNN의 SNS 언급량 예측 MSE: ', dnn_sns_mse)
    print('DNN의 SNS 언급량 예측 RMSE: ', np.sqrt(dnn_sns_mse))
    print('DNN의 SNS 언급량 예측 MAE: ', dnn_sns_mae)

Epoch 1/300
72/72 [==============================] - 1s 6ms/step - loss: 36424.9180 - val_loss: 33023.8984
Epoch 2/300
72/72 [==============================] - 0s 4ms/step - loss: 33838.8750 - val_loss: 32272.7988
Epoch 3/300
72/72 [==============================] - 0s 4ms/step - loss: 31430.7578 - val_loss: 31307.7793
Epoch 4/300
72/72 [==============================] - 0s 4ms/step - loss: 28964.7227 - val_loss: 30104.5098
Epoch 5/300
72/72 [==============================] - 0s 4ms/step - loss: 26351.2285 - val_loss: 28300.5938
Epoch 6/300
72/72 [==============================] - 0s 5ms/step - loss: 23639.2129 - val_loss: 25688.3652
Epoch 7/300
72/72 [==============================] - 0s 5ms/step - loss: 20798.7051 - val_loss: 22136.4062
Epoch 8/300
72/72 [==============================] - 0s 4ms/step - loss: 17932.8398 - val_loss: 18016.6660
Epoch 9/300
72/72 [==============================] - 0s 5ms/step - loss: 15286.2959 - val_loss: 13874.0869
Epoch 10/300
72/72 [=================

In [ ]:
dnn.save('../data/dnn_sns.h5')

In [21]:
dnn = keras.models.load_model('../data/dnn_sns.h5')

dnn_sns_pred = dnn.predict(X_test)

dnn_sns_mse = mean_squared_error(dnn_sns_pred, y_test)
dnn_sns_mae = mean_absolute_error(dnn_sns_pred, y_test)
print('DNN의 SNS 언급량 예측 MSE: ', dnn_sns_mse)
print('DNN의 SNS 언급량 예측 RMSE: ', np.sqrt(dnn_sns_mse))
print('DNN의 SNS 언급량 예측 MAE: ', dnn_sns_mae)


DNN의 SNS 언급량 예측 MSE:  650.9297557097605
DNN의 SNS 언급량 예측 RMSE:  25.513325061813493
DNN의 SNS 언급량 예측 MAE:  11.936580413659698


# 4) LSTM


## (1) 연령, 성별 카테고리별 총 판매량 예측

In [ ]:
# 상품 목록을 정의합니다. 
goods = wea_sale_ohe.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

# 상품 및 날짜로 데이터를 정렬합니다.
wea_sale_ohe.sort_values(goods+['날짜'], inplace=True)

# 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
wea_sale_ohe.index = range(0, len(wea_sale_ohe))

# 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.

weather_sc = MinMaxScaler(feature_range = (0, 1)) 
weather_scaled = weather_sc.fit_transform(wea_sale_ohe.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

# 반드시 판매량 데이터를 마지막 열에 두어야 합니다.
# scale된 데이터를 병합해서 total data를 생성합니다.
# 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
total = np.hstack([wea_sale_ohe.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                   weather_scaled,
                   wea_sale_ohe.loc[:,'20대 여성 판매량(개)':'60대 남성 판매량(개)'].values])

# 가열식 가습기 판매 데이터 개수의 80% 입니다.
index =int(len(wea_sale_ohe[wea_sale_ohe['소분류_가열식 가습기']==1]) * 0.8)

# 가스온수기 판매 데이터의 70%를 train data로, 30%를 test data로 정의합니다.
train = total[wea_sale_ohe['소분류_가열식 가습기']==1][:index]
test = total[wea_sale_ohe['소분류_가열식 가습기']==1][index:]

# 전체 상품 목록 리스트
goods = wea_sale_ohe.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

# 각 상품의 판매 데이터의 70%는 train, 30%는 test data에 추가합니다. 
for good in goods[1:]:
  index =int(len(wea_sale_ohe[wea_sale_ohe[good]==1]) * 0.8)
  train = np.vstack([train, total[wea_sale_ohe[good]==1][:index]])
  test = np.vstack([test, total[wea_sale_ohe[good]==1][index:]])

# test 데이터를 생성합니다.
X_test = []
y_test = []

day = 20

for good_index in range(0, 126):
  df = test[test[:, good_index]==1]
  for i in range(day, df.shape[0]+1):
      X_test.append(df[i-day:i, :-10])
      y_test.append(df[i-1, -10:])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], -1))

In [ ]:
path = '../data'
path = '/content/drive/MyDrive/final weather/data/'

lstm_sale = keras.models.load_model(path+'lstm_sale_80%.h5')

lstm_sale_pred = lstm_sale.predict(X_test)

lstm_sale_mse = mean_squared_error(lstm_sale_pred, y_test)
lstm_sale_mae = mean_absolute_error(lstm_sale_pred, y_test)
print('LSTM의 SALE 언급량 예측 MSE: ', lstm_sale_mse)
print('LSTM의 SALE 언급량 예측 RMSE: ', np.sqrt(lstm_sale_mse))
print('LSTM의 SALE 언급량 예측 MAE: ', lstm_sale_mae)

LSTM의 SALE 언급량 예측 MSE:  414.3808501019057
LSTM의 SALE 언급량 예측 RMSE:  20.356346678662792
LSTM의 SALE 언급량 예측 MAE:  7.179835102577077


## (2) SNS 언급량 예측

In [ ]:
# 상품 목록을 정의합니다. 
goods = wea_sns_ohe.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

# 상품 및 날짜로 데이터를 정렬합니다.
wea_sns_ohe.sort_values(goods+['날짜'], inplace=True)

# 정렬로 뒤죽박죽이 된 행 인덱스를 초기화합니다. 
wea_sns_ohe.index = range(0, len(wea_sns_ohe))

# 모델의 성능 향상을 위해 MinMaxScale을 진행합니다.

weather_sc = MinMaxScaler(feature_range = (0, 1)) 
weather_scaled = weather_sc.fit_transform(wea_sns_ohe.loc[:,'평균기온(°C)':'미세먼지(PM10)'].values)

# 반드시 언급량 데이터를 마지막 열에 두어야 합니다.
# scale된 데이터를 병합해서 total data를 생성합니다.
# 상품 소분류 원핫인코딩 열은 MinMaxScale을 진행하지 않았습니다. (0과 1로 이루어졌으므로)
total = np.hstack([wea_sns_ohe.loc[:,'소분류_가열식 가습기':'소분류_히터'].values,
                   weather_scaled,
                   wea_sns_ohe['SNS언급량'].values.reshape(-1,1)])

# 가열식 가습기 판매 데이터 개수의 80% 입니다.
index =int(len(wea_sns_ohe[wea_sns_ohe['소분류_가열식 가습기']==1]) * 0.8)

# 가스온수기 판매 데이터의 80%를 train data로, 20%를 test data로 정의합니다.
train = total[wea_sns_ohe['소분류_가열식 가습기']==1][:index]
test = total[wea_sns_ohe['소분류_가열식 가습기']==1][index:]

# 전체 상품 목록 리스트
goods = wea_sns_ohe.loc[:,'소분류_가열식 가습기':'소분류_히터'].columns.tolist()

# 각 상품의 판매 데이터의 80%는 train, 20%는 test data에 추가합니다. 
for good in goods[1:]:
  index =int(len(wea_sns_ohe[wea_sns_ohe[good]==1]) * 0.8)
  train = np.vstack([train, total[wea_sns_ohe[good]==1][:index]])
  test = np.vstack([test, total[wea_sns_ohe[good]==1][index:]])

# test 데이터도 위와 같이 생성합니다.

X_test = []
y_test = []

day = 20

for good_index in range(0, 126):
  df = test[test[:, good_index]==1]
  for i in range(day, df.shape[0]+1):
      X_test.append(df[i-day:i, :-1])
      y_test.append(df[i-1, -1])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], -1))

In [ ]:
path = '../data'
path = '/content/drive/MyDrive/final weather/data/'

lstm_sns = keras.models.load_model(path+'lstm_sns_80%.h5')

lstm_sns_pred = lstm_sns.predict(X_test)

lstm_sns_mse = mean_squared_error(lstm_sns_pred, y_test)
lstm_sns_mae = mean_absolute_error(lstm_sns_pred, y_test)
print('LSTM의 SALE 언급량 예측 MSE: ', lstm_sns_mse)
print('LSTM의 SALE 언급량 예측 RMSE: ', np.sqrt(lstm_sns_mse))
print('LSTM의 SALE 언급량 예측 MAE: ', lstm_sns_mae)

LSTM의 SALE 언급량 예측 MSE:  472.2663391315426
LSTM의 SALE 언급량 예측 RMSE:  21.731689744047575
LSTM의 SALE 언급량 예측 MAE:  8.84897483661852


#5) Ensemble(LSTM, LGBM, DNN)

LSTM으로 예측한 값을 가져와서 LGBM, DNN 결과와 ensemble하여 MSE, RMSE 값 도출

In [ ]:
print('LGBMRegressor의 판매량 예측 MSE: ', lgbm_sale_mse)
print('LGBMRegressor의 판매량 예측 RMSE: ', np.sqrt(lgbm_sale_mse))
print('LGBMRegressor의 판매량 예측 MAE: ', lgbm_sale_mae)

print('CatBoostRegressor의 판매량 예측 MSE: ', cb_sale_mse)
print('CatBoostRegressor의 판매량 예측 RMSE: ', np.sqrt(cb_sale_mse))
print('CatBoostRegressor의 판매량 예측 MAE: ', cb_sale_mae)

print('DNN의 판매량 예측 MSE: ', dnn_sale_mse)
print('DNN의 판매량 예측 RMSE: ', np.sqrt(dnn_sale_mse))
print('DNN의 판매량 예측 MAE: ', dnn_sale_mae)

lstm_sale_mse = mean_squared_error(lstm_sale_pred, y_test)
lstm_sale_mae = mean_absolute_error(lstm_sale_pred, y_test)
print('LSTM의 판매량 예측 MSE: ', lstm_sale_mse)
print('LSTM의 판매량 예측 RMSE: ', np.sqrt(lstm_sale_mse))
print('LSTM의 판매량 예측 MAE: ', lstm_sale_mae)

ensemble_sale_mse = mean_squared_error((lstm_sale_pred + cb_sale_pred +lgbm_sale_pred + dnn_sale_pred)/4, y_test)
ensemble_sale_mae = mean_absolute_error((lstm_sale_pred +cb_sale_pred + lgbm_sale_pred + dnn_sale_pred)/4, y_test)

print('Ensemble Model의 판매량 예측 MSE:', ensemble_sale_mse)
print('Ensemble Model의 판매량 예측 RMSE:', np.sqrt(ensemble_sale_mse))
print('Ensemble Model의 판매량 예측 MAE:', ensemble_sale_mae)

LGBMRegressor의 판매량 예측 MSE:  439.5142314168567
LGBMRegressor의 판매량 예측 RMSE:  20.96459471148576
LGBMRegressor의 판매량 예측 MAE:  7.958411629060916
CatBoostRegressor의 판매량 예측 MSE:  437.20838539796597
CatBoostRegressor의 판매량 예측 RMSE:  20.909528579046587
CatBoostRegressor의 판매량 예측 MAE:  7.981197987829626
DNN의 판매량 예측 MSE:  441.1367906315193
DNN의 판매량 예측 RMSE:  21.003256667277086
DNN의 판매량 예측 MAE:  7.8051662188420465
LSTM의 판매량 예측 MSE:  414.3808501019057
LSTM의 판매량 예측 RMSE:  20.356346678662792
LSTM의 판매량 예측 MAE:  7.179835102577077
Ensemble Model의 판매량 예측 MSE: 405.36376750816885
Ensemble Model의 판매량 예측 RMSE: 20.133647645376357
Ensemble Model의 판매량 예측 MAE: 7.322144338831313


In [ ]:
X_train, X_test, y_train, y_test = sns_dataset(wea_sns_ohe)

In [ ]:
print('LGBMRegressor의 SNS 언급량 예측 MSE: ', lgbm_sns_mse)
print('LGBMRegressor의 SNS 언급량 예측 RMSE: ', np.sqrt(lgbm_sns_mse))
print('LGBMRegressor의 SNS 언급량 예측 MAE: ', lgbm_sns_mae)

print('CatBoostRegressor의 SNS 언급량 예측 MSE:', cb_sns_mse)
print('CatBoostRegressor의 SNS 언급량 예측 RMSE:', np.sqrt(cb_sns_mse))
print('CatBoostRegressor의 SNS 언급량 예측 MAE:', cb_sns_mae)

print('DNN의 SNS 언급량 예측 MSE: ', dnn_sns_mse)
print('DNN의 SNS 언급량 예측 RMSE: ', np.sqrt(dnn_sns_mse))
print('DNN의 SNS 언급량 예측 MAE: ', dnn_sns_mae)

lstm_sns_mse = mean_squared_error(lstm_sns_pred, y_test)
lstm_sns_mae = mean_absolute_error(lstm_sns_pred, y_test)
print('LSTM의 SNS 언급량 예측 MSE: ', lstm_sns_mse)
print('LSTM의 SNS 언급량 예측 RMSE: ', np.sqrt(lstm_sns_mse))
print('LSTM의 SNS 언급량 예측 MAE: ', lstm_sns_mae)

ensemble_sns_mse = mean_squared_error((lstm_sns_pred.reshape(-1,1) + cb_sns_pred.reshape(-1,1) + lgbm_sns_pred.reshape(-1,1) + dnn_sns_pred.reshape(-1,1))/4, y_test)
ensemble_sns_mae = mean_absolute_error((lstm_sns_pred.reshape(-1,1) + cb_sns_pred.reshape(-1,1) + lgbm_sns_pred.reshape(-1,1) + dnn_sns_pred.reshape(-1,1))/4, y_test)

print('Ensemble Model의 SNS 언급량 예측 MSE:', ensemble_sns_mse)
print('Ensemble Model의 SNS 언급량 예측 RMSE:', np.sqrt(ensemble_sns_mse))
print('Ensemble Model의 SNS 언급량 예측 MAE:', ensemble_sns_mae)

LGBMRegressor의 SNS 언급량 예측 MSE:  674.1798275828507
LGBMRegressor의 SNS 언급량 예측 RMSE:  25.964973090354835
LGBMRegressor의 SNS 언급량 예측 MAE:  10.479638969313406
CatBoostRegressor의 SNS 언급량 예측 MSE: 650.9868435318208
CatBoostRegressor의 SNS 언급량 예측 RMSE: 25.514443821722253
CatBoostRegressor의 SNS 언급량 예측 MAE: 10.524364601335625
DNN의 SNS 언급량 예측 MSE:  650.9296769590771
DNN의 SNS 언급량 예측 RMSE:  25.513323518488868
DNN의 SNS 언급량 예측 MAE:  11.936579886179823
LSTM의 SNS 언급량 예측 MSE:  472.2663391315426
LSTM의 SNS 언급량 예측 RMSE:  21.731689744047575
LSTM의 SNS 언급량 예측 MAE:  8.84897483661852
Ensemble Model의 SNS 언급량 예측 MSE: 552.0237775878632
Ensemble Model의 SNS 언급량 예측 RMSE: 23.49518626416618
Ensemble Model의 SNS 언급량 예측 MAE: 9.719405011403317
